In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [19]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [20]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nash_nppes
"""

nash_nppes = pd.read_sql(query,db)

In [21]:
db.close()

In [22]:
nash_nppes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38148 entries, 0 to 38147
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  38148 non-null  int64  
 1   entity_type_code     38148 non-null  float64
 2   org_name             7466 non-null   object 
 3   last_name            30681 non-null  object 
 4   first_name           30682 non-null  object 
 5   name_suffix          895 non-null    object 
 6   provider_credential  25175 non-null  object 
 7   address_1            38148 non-null  object 
 8   zip                  38148 non-null  object 
 9   taxonomy_code        38148 non-null  object 
 10  code                 38148 non-null  object 
 11  grouping             38148 non-null  object 
 12  classification       38148 non-null  object 
 13  specialization       15120 non-null  object 
dtypes: float64(1), int64(1), object(12)
memory usage: 4.1+ MB


In [23]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT COUNT(*)
    FROM hop_team
"""

hop_team = pd.read_sql(query,db)

db.close()

In [24]:
hop_team

,COUNT(*)
0,31704890


### How many records in the hop_team table overlap with NPI numbers in nash_nppes?

In [34]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
   SELECT *
    FROM nash_nppes
    WHERE entity_type_code == 2  
"""

test = pd.read_sql(query,db)

db.close()

In [35]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7466 entries, 0 to 7465
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   npi                  7466 non-null   int64  
 1   entity_type_code     7466 non-null   float64
 2   org_name             7466 non-null   object 
 3   last_name            0 non-null      object 
 4   first_name           0 non-null      object 
 5   name_suffix          0 non-null      object 
 6   provider_credential  0 non-null      object 
 7   address_1            7466 non-null   object 
 8   zip                  7466 non-null   object 
 9   taxonomy_code        7466 non-null   object 
 10  code                 7466 non-null   object 
 11  grouping             7466 non-null   object 
 12  classification       7466 non-null   object 
 13  specialization       2789 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 816.7+ KB


In [36]:
#Checking Number of Records 
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT COUNT(*)
    FROM hop_team
    WHERE from_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes
        WHERE entity_type_code == 1)    
    AND to_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes
        WHERE entity_type_code ==2)
"""

hops = pd.read_sql(query,db)

db.close()

In [ ]:
#query = """
#    SELECT *
#    FROM hop_team
#    WHERE from_npi IN(
#        SELECT DISTINCT(npi)
#        FROM nash_nppes)
#    AND to_npi IN(
#        SELECT DISTINCT(npi)
#        FROM nash_nppes)
#"""

In [37]:
hops

,COUNT(*)
0,41127


There are 214,316 records in hop_team where the 'from_npi' NPI code matches one in the nash_nppes table

There are 214,080 records in hop_team where the 'to_npi' NPI code matches one in the nash_nppes table

In total, there are 138,493 hop_team records where both the 'to' and 'from' NPIs are in the nash_nppes table

If we filter down so that BOTH to/from npi are in Nashville and the from_npi to be entity type 1 and to_npi to be entity type 2, then there are 41,127 records.

### Add the filtered table to the database as 'nash_hops'

In [43]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM hop_team
    WHERE from_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes
        WHERE entity_type_code == 1)
    AND to_npi IN(
        SELECT DISTINCT(npi)
        FROM nash_nppes
        WHERE entity_type_code ==2)
"""

nash_hops = pd.read_sql(query,db)

db.close()

In [44]:
nash_hops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41127 entries, 0 to 41126
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   from_npi           41127 non-null  int64  
 1   to_npi             41127 non-null  int64  
 2   patient_count      41127 non-null  int64  
 3   transaction_count  41127 non-null  int64  
 4   average_day_wait   41127 non-null  float64
 5   std_day_wait       41127 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 1.9 MB


In [45]:
#db = sqlite3.connect('../data/nppes_lite.sqlite') #open connection

#nash_hops.to_sql('nash_hops', db, if_exists = 'append', index = False) #save nash_hops data

#db.close() #close connection

In [2]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nash_hops
"""

test = pd.read_sql(query,db)

db.close()
test

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1023055126,1003013947,69,142,5.535,27.885
1,1013154723,1003028770,32,50,41.940,52.320
2,1003963976,1003028770,2535,3945,0.000,0.000
3,1023055126,1003031261,69,99,37.778,69.205
4,1013109511,1003075862,112,360,23.950,19.562
...,...,...,...,...,...,...
179615,1255575320,1245485549,914,958,4.653,29.739
179616,1235137183,1245485549,198,253,19.470,43.328
179617,1245541846,1245485549,95,111,29.559,52.417
179618,1255428389,1245485549,191,236,5.004,26.636


In [3]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nash_nppes
"""

test = pd.read_sql(query,db)

db.close()
test

,npi,entity_type_code,org_name,last_name,first_name,name_suffix,provider_credential,address_1,zip,taxonomy_code,code,grouping,classification,specialization
0,1134122187,1.0,None,RUDNICKE,CHERYL,None,CRNP,250 25TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
1,1043298722,1.0,None,CARO,SUSAN,None,"MSN, RNC, APNG",300 20TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
2,1255310058,1.0,None,RUBLE,MAMIE,None,N.P.,3310 WEST END AVE,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
3,1740232081,1.0,None,BEADLE,TRACY,None,None,526 8TH AVE S,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
4,1962445585,1.0,None,GREEN-HADDEN,JOY,None,NP,2400 PATTERSON ST,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38143,1659714087,1.0,None,TASHJIAN,RANDY,None,MD,850 RS GASS BLVD,37216,207ZF0201X,207ZF0201X,Allopathic & Osteopathic Physicians,Pathology,Forensic Pathology
38144,1730671223,1.0,None,LEWIS,ROGER,None,None,717 HART LN,37216,225C00000X,225C00000X,"Respiratory, Developmental, Rehabilitative and...",Rehabilitation Counselor,None
38145,1376751099,2.0,"FRANK T RUTHERFORD MEMORIAL HOSPITAL, INC",None,None,None,None,126 JMZ DR,38563,261QC0050X,261QC0050X,Ambulatory Health Care Facilities,Clinic/Center,Critical Access Hospital
38146,1992223804,1.0,None,RAY,TINA,None,None,333 COMMERCE ST STE 700,37201,364SG0600X,364SG0600X,Physician Assistants & Advanced Practice Nursi...,Clinical Nurse Specialist,Gerontology


### Test join nash_nppes info to nash_hops for just from_npi

In [4]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nash_nppes
    LIMIT 10
"""

test = pd.read_sql(query,db)

db.close()

In [5]:
test

,npi,entity_type_code,org_name,last_name,first_name,name_suffix,provider_credential,address_1,zip,taxonomy_code,code,grouping,classification,specialization
0,1134122187,1.0,None,RUDNICKE,CHERYL,None,CRNP,250 25TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
1,1043298722,1.0,None,CARO,SUSAN,None,"MSN, RNC, APNG",300 20TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
2,1255310058,1.0,None,RUBLE,MAMIE,None,N.P.,3310 WEST END AVE,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
3,1740232081,1.0,None,BEADLE,TRACY,None,None,526 8TH AVE S,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
4,1962445585,1.0,None,GREEN-HADDEN,JOY,None,NP,2400 PATTERSON ST,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
5,1639199649,1.0,None,PETERSON,CHRISTINE,None,NP,832 WEDGEWOOD AVE,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
6,1093736720,1.0,None,JACKSON,TAMMY,None,NP,300 20TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
7,1548274731,1.0,None,ANTANAITIS,MICHAEL,None,P.A.,1501 12TH AVE S,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
8,1356434864,1.0,None,HURTIG,JANE,None,NP,300 20TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None
9,1447344668,1.0,None,HRONEK,JAN,None,NP,250 25TH AVE N,37203,363L00000X,363L00000X,Physician Assistants & Advanced Practice Nursi...,Nurse Practitioner,None


### Examine taxonomy

In [2]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
    SELECT *
    FROM nucc_taxonomy
"""

taxonomy = pd.read_sql(query,db)

db.close()

In [8]:
taxonomy.grouping.unique()

array(['Group', 'Allopathic & Osteopathic Physicians',
       'Behavioral Health & Social Service Providers',
       'Chiropractic Providers', 'Dental Providers',
       'Dietary & Nutritional Service Providers',
       'Emergency Medical Service Providers',
       'Eye and Vision Services Providers', 'Nursing Service Providers',
       'Nursing Service Related Providers', 'Other Service Providers',
       'Pharmacy Service Providers',
       'Physician Assistants & Advanced Practice Nursing Providers',
       'Podiatric Medicine & Surgery Service Providers',
       'Respiratory, Developmental, Rehabilitative and Restorative Service Providers',
       'Speech, Language and Hearing Service Providers',
       'Student, Health Care',
       'Technologists, Technicians & Other Technical Service Providers',
       'Agencies', 'Ambulatory Health Care Facilities', 'Hospital Units',
       'Hospitals', 'Laboratories', 'Managed Care Organizations',
       'Nursing & Custodial Care Facilities',


Exclude these? 

Under Grouping: 
        Laboratories, 
        Suppliers(? DME, Pharmacies, Prosthetic/Orthotic etc.), 
        Transportation Services,
        Group,
        
?Dental

Under Classification: Pharmacy

In [17]:
taxonomy[taxonomy.grouping=='Dental Providers']

,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
281,125K00000X,Dental Providers,Advanced Practice Dental Therapist,None,An Advanced Practice Dental Therapist is:<br><...,10/1/2012,None,None,Source: Summarized from Minnesota Statute 150A...,Advanced Practice Dental Therapist
282,126800000X,Dental Providers,Dental Assistant,None,An individual who may or may not have complete...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Assistant
283,124Q00000X,Dental Providers,Dental Hygienist,None,An individual who has completed an accredited ...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Hygienist
284,126900000X,Dental Providers,Dental Laboratory Technician,None,An individual who has the skill and knowledge ...,4/1/2002,None,None,Source: Comprehensive Policy Statement on Dent...,Dental Laboratory Technician
285,125J00000X,Dental Providers,Dental Therapist,None,A Dental Therapist is an individual who has co...,10/1/2012,None,None,Source: Summarized from Minnesota Statute 150A...,Dental Therapist
286,122300000X,Dental Providers,Dentist,None,A dentist is a person qualified by a doctorate...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Dentist
287,1223D0001X,Dental Providers,Dentist,Dental Public Health,The science and art of preventing and controll...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Public Health Dentist
288,1223D0004X,Dental Providers,Dentist,Dentist Anesthesiologist,A dentist who has successfully completed an ac...,4/1/2013,None,None,Source: The American Society of Dentist Anesth...,Dentist Anesthesiologist
289,1223E0200X,Dental Providers,Dentist,Endodontics,The branch of dentistry that is concerned with...,4/1/2002,None,None,Source: Council on Dental Education and Licens...,Endodontist
290,1223G0001X,Dental Providers,Dentist,General Practice,A general dentist is the primary dental care p...,4/1/2002,None,None,Source: Academy of General Dentistry,General Practice Dentistry


In [ ]:
db = sqlite3.connect('../data/nppes_lite.sqlite') #reopen the connection

query = """
SELECT 
    from_npi,
    
FROM nash_hops
"""

taxonomy = pd.read_sql(query,db)

db.close()